#### Chroma DB
https://docs.langchain.com/oss/python/integrations/vectorstores#chroma

In [2]:
# perform similarity search using ollama Embeddings and Chroma vector store DB

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader("speech.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="mxbai-embed-large")

db = Chroma.from_documents(
    documents=texts, embedding=embeddings, collection_name="speech_collection"
)
query = "What is the main message of the speech?"
results = db.similarity_search(query=query, k=3)

print(f"Top 3 results for the query: '{query}'")
for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

w:\krish_naik\python\AI_Bootcamp\4.Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\dennis joshua j\AppData\Local\Temp\ipykernel_19324\727102273.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large")


Top 3 results for the query: 'What is the main message of the speech?'
Result 1: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 2: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.



##### Save embeddings to local disk

In [3]:
db = Chroma.from_documents(
    documents=texts, embedding=embeddings, persist_directory="./chroma11_db"
)

##### Load from local disk

In [4]:
db2 = Chroma(persist_directory='./chroma11_db', embedding_function=embeddings)
query = "What is the main message of the speech?"
results = db2.similarity_search(query=query, k=3)

print(f"Top 3 results for the query: '{query}'")
for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

C:\Users\dennis joshua j\AppData\Local\Temp\ipykernel_19324\2646031743.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db2 = Chroma(persist_directory='./chroma11_db', embedding_function=embeddings)


Top 3 results for the query: 'What is the main message of the speech?'
Result 1: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 3: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.



##### Perform similarity search with score
Returned score:
- lower = better (distance)
- 0.0 ≈ perfect match

In [6]:
docs_with_score = db.similarity_search_with_score(query)
docs_with_score

[(Document(metadata={'source': 'speech.txt'}, page_content='The present asks us to build with integrity and compassion.\nThe future depends on how wisely we use our freedom today.\nIndependence lives on when we choose progress over fear.'),
  0.7931689023971558),
 (Document(metadata={'source': 'speech.txt'}, page_content='Freedom was not gifted; it was earned through courage and sacrifice.\nCountless voices rose together to demand dignity and self-rule.\nEvery step toward independence carried the weight of hope and loss.\nThe struggle taught us unity beyond language, region, or belief.\nIndependence is not just a date, but a responsibility we carry daily.\nIt reminds us to protect justice, equality, and truth.\nThe past whispers lessons of resilience and bravery.'),
  0.7961559891700745)]

##### Converting vector stores to retreiver class (important)

In [5]:
retriever = db.as_retriever(search_kwargs={"k": 3})
results = retriever.invoke(query)

for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 3: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.



##### MMR — Max Marginal Relevance (very important)
- prevents multiple chunks from saying the same thing.
- fetch_k : initial candidates
- k : final candidates
- fetch_k = 3x or 5x of k

In [6]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10}
)
results = retriever.invoke(query)

for i, res in enumerate(results):
    print(f"Result {i+1}: {res.page_content}\n")

Result 1: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 2: The present asks us to build with integrity and compassion.
The future depends on how wisely we use our freedom today.
Independence lives on when we choose progress over fear.

Result 3: Freedom was not gifted; it was earned through courage and sacrifice.
Countless voices rose together to demand dignity and self-rule.
Every step toward independence carried the weight of hope and loss.
The struggle taught us unity beyond language, region, or belief.
Independence is not just a date, but a responsibility we carry daily.
It reminds us to protect justice, equality, and truth.
The past whispers lessons of resilience and bravery.

